In [1]:
import torch
import glob

import numpy as np
import torch.nn as nn

from tqdm import tqdm

In [2]:
# We highly recommend to use a GPU for train and use this model.

device = "cuda" if torch.cuda.is_available() else "cpu"

### Load the previously trained model

In [3]:
# Define the CNN architecture
scale_channel   = 1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.input    = nn.Sequential(
            # 1                                             #3x128x128
            nn.Conv2d(3, 12*scale_channel, kernel_size=3),  #12x128x128
            nn.Tanh(),
            nn.ConvTranspose2d(12*scale_channel, 12*scale_channel, kernel_size=3),
            nn.Tanh()
        )

        self.hidden_1   = nn.Sequential(
            # 2
            nn.ConvTranspose2d(12*scale_channel, 12*scale_channel, kernel_size=49), #12x176x176
            nn.Tanh(),
            nn.Conv2d(12*scale_channel, 12*scale_channel, kernel_size=49),  #12x128x128
            nn.Tanh()
        )

        self.hidden_2   = nn.Sequential(
            # 3
            nn.ConvTranspose2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh(),
            nn.Conv2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh()
        )

        self.hidden_3   = nn.Sequential(
            # 4
            nn.ConvTranspose2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh(),
            nn.Conv2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh()
        )

        self.hidden_4   = nn.Sequential(
            # 5
            nn.ConvTranspose2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh(),
            nn.Conv2d(12*scale_channel, 12*scale_channel, kernel_size=49),
            nn.Tanh()
        )

        self.output     = nn.Sequential(
            # 6
            nn.Conv2d(12*scale_channel, 1, kernel_size=1)
        )

    def forward(self, x):
        x1  = self.input(x)
        
        x2  = self.hidden_1(x1)
        x2  = torch.mul(x2, x1)

        x3  = self.hidden_2(x2)
        x3  = torch.mul(x3, x2)

        x4  = self.hidden_3(x3)
        x4  = torch.mul(x4, x3)

        x5  = self.hidden_4(x4)
        x5  = torch.mul(x5, x3)

        outputs = self.output(x5)

        return outputs
    
# Prepare our model to implement the given data for train
model   = Net()
model   = model.to(device=device)

In [ ]:
# Load a previous model for a initial state model

if device == "cuda":
    trained_model = torch.load("checkpoint.pth")
else:
    trained_model = torch.load('checkpoint.pth', map_location=torch.device('cpu'))

model.load_state_dict(trained_model)

##### To rank our model into the competition, we download the validation data

In [ ]:
# Download the validation data. Don't forget to decompress the folder

url = ("https://descargas.inf.santiago.usm.cl/test_public.tar.gz")
print(url)

In [ ]:
# Load the test_public to create the validation data

file_validation = np.array([])

valid_label = glob.glob(f"test_public/EPSILON/*.npy")
valid_label = np.append(file_validation, valid_label)
valid_label.sort()

valid_input = np.array([np.load(valid_label[n]) for n in tqdm(range(len(valid_label)))])

In [ ]:
# Save the results in another folder for submit the results

validation_number   = valid_input.shape[0]

for n in tqdm(range(validation_number)):
    tensor_input = torch.Tensor(valid_input[n])
    tensor_input = tensor_input.to(device=device)

    output  = model(tensor_input)[0]
    output  = output.to(device = "cpu")
    output  = output.detach().numpy()
    label   = valid_label[n]
    label   = label[20:]
    
    np.save('qbirds/'+label, output)